<a href="https://colab.research.google.com/github/An-Dongsun/Section2-Project/blob/main/AI_13_%EC%95%88%EB%8F%99%EC%84%A0_section2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HeartDisease 예측모델 Project

# 🎯 해결하고자 하는 문제
Main : 새로운 환자의 설문조사 결과를 바탕으로 심장병의 유무를 파악한다.   
Sub :  심장병에 악영향을 미치는 요인과 심장병을 예방하는 요인을 파악 하고싶다.

# 선정 이유
* 한국인의 사망 원인 1위인 암에 이어 2위인 심장병은 (2010정도 까지 사망원인 2위 였던 뇌혈관질환은 감소한 반면) 지속적으로 상승하고 있어서 위험하다고 판단이 되고 있고 미국에서도 대부분의 인종의 주요 사망 원인 중 하나이다.(모든 사람에게 위험한 질환이다.)

* 3대 사인(암, 심장 질환, 폐렴)은 2020년 기준 전체 사인의 44.9%를 차지하고 있다. → 순환 계통 질환의 사망률 전년 대비 3.2% 증가

* 심장병의 유무를 판단해서 조기에 발견한다면 적절한 관리를 통해 사망확률이 낮아질 수 있는 질병이다.

* 따라서 심장병을 조기에 발견하고 악영향을 미치는 요인을 감지하고 예방하는 것은 의료에서 매우 중요하다.

# 필요한 라이브러리 설치 및 불러오기
* 설치한 후 런타임 초기화 한번 해주어야 한다.

In [4]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install markupsafe == 2.0.1
    !pip install --upgrade pandas-profiling
    !pip install pdpbox
    !pip install shap

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost
import eli5
import shap

import warnings

from markupsafe import escape
from pandas_profiling import ProfileReport

from ipywidgets import interact, fixed

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

from sklearn.pipeline import make_pipeline, Pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder, TargetEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, plot_confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve

from sklearn.inspection import permutation_importance

from eli5.sklearn import PermutationImportance
from pdpbox.pdp import pdp_isolate, pdp_interact, pdp_interact_plot, pdp_plot

warnings.filterwarnings(action='ignore')

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


# 1. 데이터셋 불러오기

In [6]:
#깃허브에 저장되어 있는 데이터 파일을 불러온다.
heart = pd.read_csv('https://raw.githubusercontent.com/An-Dongsun/Section2-Project/main/heart_2020_cleaned.csv')

#전부 잘 불러와졌는지 확인한다.
heart.shape

(319795, 18)

In [7]:
#맨 위 5행을 불러와서 확인한다.
heart.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [8]:
#결측치의 갯수를 확인한다.
pd.DataFrame(heart.isnull().sum(), columns=["결측치 개수"])

,결측치 개수
HeartDisease,0
BMI,0
Smoking,0
AlcoholDrinking,0
Stroke,0
PhysicalHealth,0
MentalHealth,0
DiffWalking,0
Sex,0
AgeCategory,0


# 2. feature 설명
* HeartDisease: 관상 동맥 심장 질환(CHD) 또는 심근 경색증(MI)이 있다고 보고한 적이 있는 응답자 (Y/N) <- target
* BMI: 체질량 지수(BMI)
* Smoking: 당신은 일생 동안 적어도 100개비의 담배를 피웠습니까? (Y/N) [참고: 5갑 = 100개비]
* AlcoholDrinking: 과음자(성인 남성은 주당 14잔 이상, 성인 여성은 주당 7잔 이상 음주) (Y/N)
* Stroke: (당신은) 뇌졸중이 있습니까? (Y/N)
* PhysicalHealth: 귀하의 신체적 질병과 부상을 포함한 신체적 건강을 생각해보면, 지난 30일 동안 귀하의 건강이 좋지 않았던 날은 며칠입니까? (0-30일)
* MentalHealth: 귀하의 정신건강을 생각해보면, 지난 30일 동안 귀하의 정신건강이 좋지 않은 날은 며칠입니까? (0-30일)
* DiffWalking: 걷거나 계단을 오르는데 심각한 어려움이 있습니까? (Y/N)
* Sex: 성별 ('Female', 'Male')
* AgeCategory: 연령범주 ('18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80 or older')
* Race: 인종 ('White', 'Black', 'Asian', 'American Indian/Alaskan Native', 'Hispanic', 'Other')
* Diabetic: (당신은) 당뇨병이 있습니까? ('Yes', 'No', 'No, borderline diabetes', 'Yes (during pregnancy)')
* PhysicalActivity: 지난 30일 동안 정규직 이외의 신체활동이나 운동을 했다. (Y/N)
* GenHealth: 일반적으로 당신의 건강은 어떤가? ('Excellent', 'Very good', 'Good', 'Fair', 'Poor')
* SleepTime : 귀하는 24시간 중 평균 몇 시간의 수면을 취하십니까? (1 ~ 24)
* Asthma: (당신은) 천식이 있습니까? (Y/N)
* KidneyDisease: 신장 결석, 방광 감염 또는 요실금을 제외하고 신장 질환이 있다는 말을 들은 적이 있습니까? (Y/N)
* SkinCancer: (당신은) 피부암이 있습니까? (Y/N)

In [23]:
# 각 feature의 고유한 값의 갯수를 구한다.
heart.nunique()

HeartDisease           2
BMI                 3604
Smoking                2
AlcoholDrinking        2
Stroke                 2
PhysicalHealth        31
MentalHealth          31
DiffWalking            2
Sex                    2
AgeCategory           13
Race                   6
Diabetic               4
PhysicalActivity       2
GenHealth              5
SleepTime             24
Asthma                 2
KidneyDisease          2
SkinCancer             2
dtype: int64

In [24]:
# 각 feature의 설문 답변을 구하는 코드이다.
heart['Diabetic'].unique()

array(['Yes', 'No', 'No, borderline diabetes', 'Yes (during pregnancy)'],
      dtype=object)